In [1]:
import pandas as pd
import numpy as np
import random as rnd

import seaborn as sns
import matplotlib.pyplot as plt

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

#reading data
traindata = pd.read_csv('all/train.csv')
testdata = pd.read_csv('all/test.csv')

traindata = traindata.drop(['Ticket','Cabin'],axis=1)
testdata = testdata.drop(['Ticket','Cabin'], axis=1)
combine = [traindata,testdata]

mapper = {'Mr':1, 'Miss':2, 'Mrs':3, 'Master':4, 'Rare':5}
for data in combine:
    data['Title'] = data.Name.str.extract(' ([A-Za-z]+)\.',expand=False)
    data['Title'] = data['Title'].replace(['Lady','Capt','Col','Countess','Don','Jonkheer',\
                                          'Major','Rev','Sir','Dr','Dona'],'Rare')
    data['Title'] = data['Title'].replace(['Mlle','Mme','Ms'],'Miss')
    data['Title'] = data['Title'].map(mapper).fillna(0)
    data['Sex'] = data['Sex'].map({'female':1, 'male':0}).astype(int)

# print(pd.crosstab(traindata['Title'],traindata['Sex']))

# print(traindata[['Title','Survived']].groupby('Title').mean())


traindata=traindata.drop(['Name', 'PassengerId'], axis=1)
testdata=testdata.drop(['Name'], axis=1)
combine = [traindata, testdata]

guess_age = np.zeros((2,3))
for data in combine:
    for i in range(2):
        for j in range(3):
            guess_data = data[(data['Sex']==i) & (data['Pclass']==j+1)]['Age'].dropna()
            age = guess_data.median()
            guess_age[i, j] = int(2*age+1)*0.5
            
    for i in range(2):
        for j in range(3):
            data.loc[(data['Sex']==i) & (data['Pclass']==j+1) & (data.Age.isnull()),\
                     'Age'] = guess_age[i,j]
    data['Age']=data['Age'].astype(int)

for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']=4
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
    freq_port = dataset.Embarked.dropna().mode()[0]
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    dataset['Fare'].fillna(testdata['Fare'].dropna().median(), inplace=True)
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    
traindata=traindata.drop(['SibSp', 'Parch'], axis=1)
testdata=testdata.drop(['SibSp', 'Parch'], axis=1)
print(traindata[['Embarked', 'Survived']].groupby(['Embarked'],\
                                                  as_index=False).mean().sort_values(by='Survived', \
                                                                                     ascending=False))
print(traindata.head(10))

Y_train = traindata['Survived']
X_train = traindata.drop('Survived', axis=1)
X_test = testdata.drop('PassengerId',axis=1).copy()
X_train.shape, X_test.shape, Y_train.shape

   Embarked  Survived
1         1  0.553571
2         2  0.389610
0         0  0.339009
   Survived  Pclass  Sex  Age  Fare  Embarked  Title  FamilySize  IsAlone
0         0       3    0    1     0         0      1           2        0
1         1       1    1    2     3         1      3           2        0
2         1       3    1    1     1         0      2           1        1
3         1       1    1    2     3         0      3           2        0
4         0       3    0    2     1         0      1           1        1
5         0       3    0    1     1         2      1           1        1
6         0       1    0    3     3         0      1           1        1
7         0       3    0    0     2         0      4           5        0
8         1       3    1    1     1         0      3           3        0
9         1       2    1    0     2         1      3           2        0


((891, 8), (418, 8), (891,))

In [89]:
logreg = LogisticRegression(solver="lbfgs")
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train,Y_train)*100,2)
coeff_df = pd.DataFrame(traindata.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])
coeff_df.sort_values(by='Correlation', ascending=False)
print(acc_log)

81.59


In [90]:
submission = pd.DataFrame({
        "PassengerId": testdata["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('submission.csv', index=False)

import subprocess
#subprocess.check_output(['ls','-l']) #all that is technically needed...
print(subprocess.check_output(['kaggle','competitions','submit', '-c', \
                               'titanic', '-f', 'submission.csv', '-m', '"Message"']))

b'Successfully submitted to Titanic: Machine Learning from Disaster'


In [80]:
svc = SVC(C=1, kernel="poly", gamma="auto", degree=6)
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

87.54

In [32]:
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

84.4

In [5]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

79.91

In [6]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

/home/krullun/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


80.25

In [7]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

/home/krullun/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


79.57

In [8]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

88.55

In [9]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

88.55

In [10]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,88.55
7,Decision Tree,88.55
1,KNN,85.75
0,Support Vector Machines,83.39
2,Logistic Regression,81.26
5,Perceptron,80.25
4,Naive Bayes,79.91
6,Stochastic Gradient Decent,79.57


In [ ]:
g = sns.FacetGrid(traindata, col='Survived')
g.map(plt.hist, 'Age', bins=50)

# grid = sns.FacetGrid(traindata, col='Survived', row='Pclass')#, size=2.2, aspect=1.6)
# grid.map(plt.hist, 'Age', alpha=.5, bins=10)
# grid.add_legend();

# grid = sns.FacetGrid(train_df, col='Embarked')

In [ ]:
url = ('https://archive.ics.uci.edu/ml/' 'machine-learning-databases/abalone/abalone.data')
cols = ['sex', 'length', 'diam', 'height', 'weight', 'rings']
abalone = pd.read_csv(url, usecols=[0, 1, 2, 3, 4, 8], names=cols)
print(abalone)

In [ ]:
import pandas.util.testing as tm
tm.N, tm.K = 4,6
np.random.seed(444)
# tm.makeTimeDataFrame(freq='m').head()
# print(pd.Series._accessors)
pd.Series._accessors

In [ ]:
from itertools import product
datecols = ['year', 'month', 'day']

df = pd.DataFrame(list(product([2017, 2016], [1, 2], [1, 2, 3])),
                  columns=datecols)
df['data'] = np.random.randn(len(df))
# print(df)
df.index = pd.to_datetime(df[datecols])
# print(df)
a = pd.Series(df.index)
print(a.dt.weekday_name)
df.head()



In [ ]:
addr = pd.Series([
    'Washington, D.C. 20003',
    'Brooklyn, NY 11211-1755',
    'Omaha, NE 68154',
    'Pittsburgh, PA 15211'
])

print(addr.str.upper())






print(addr.str.count(r'\d'))  # 5 or 9-digit zip?

regex = (r'(?P<city>[A-Za-z ]+), '      # One or more letters
         r'(?P<state>[A-Z]{2}) '        # 2 capital letters
         r'(?P<zip>\d{5})(?:-\d{4})?')  # Optional 4-digit extension

addr.str.replace('.', '').str.extract(regex)

In [ ]:
colors = pd.Series([
    'periwinkle',
    'mint green',
    'burnt orange',
    'periwinkle',
    'burnt orange',
    'rose',
    'rose',
    'mint green',
    'rose',
    'navy'
])

mapper = {a: b for b, a in enumerate(colors.unique())}
as_int = colors.map(mapper)
print(as_int)
cc = colors.astype('category')
# cc.cat.categories
cc.cat.reorder_categories(mapper).cat.codes
cc = cc.cat.add_categories(['alpha'])
cc.iloc[5] = 'alpha'
